In [3]:
#@title Install
!pip install translate
!pip install tqdm

In [5]:
from translate import Translator
from tqdm import tqdm


def traduzir_srt(caminho_entrada, caminho_saida, src='en', dest='pt'):
    tradutor = Translator(from_lang=src, to_lang=dest)

    with open(caminho_entrada, 'r', encoding='utf-8') as arquivo:
        linhas = arquivo.readlines()

    resultado = []
    bloco_texto = []

    # Pré-processamento: conta quantos blocos existem
    blocos = []
    bloco_temp = []

    for linha in linhas:
        linha_strip = linha.strip()
        if linha_strip == '':
            if bloco_temp:
                blocos.append(list(bloco_temp))
                bloco_temp.clear()
        elif '-->' in linha_strip or linha_strip.isdigit():
            pass
        else:
            bloco_temp.append(linha_strip)
    if bloco_temp:
        blocos.append(list(bloco_temp))

    bloco_index = 0
    i = 0
    total_blocos = len(blocos)

    with tqdm(total=total_blocos, desc="Traduzindo legendas", unit="bloco") as pbar:
        while i < len(linhas):
            linha_strip = linhas[i].strip()

            if linha_strip == '':
                if bloco_index < total_blocos:
                    texto_original = ' '.join(blocos[bloco_index])
                    try:
                        traducao = tradutor.translate(texto_original)
                    except Exception as e:
                        traducao = texto_original
                        print(f"\n[Erro ao traduzir]: {e}")

                    for parte in traducao.split('. '):
                        resultado.append(parte.strip())
                    resultado.append('')
                    bloco_index += 1
                    pbar.update(1)
            elif '-->' in linha_strip or linha_strip.isdigit():
                resultado.append(linha_strip)
            i += 1

    with open(caminho_saida, 'w', encoding='utf-8') as saida:
        for linha in resultado:
            saida.write(linha.strip() + '\n')

    print('\n✅ Tradução finalizada com sucesso!')


In [6]:
Legenda_original_caminho = "/content/whisper_subtitle/Make your first Aimbot and ESP! C#⁄.NET Course Part 2 - Reversing structures (inglês_ASR).srt"

# Exemplo de uso:
traduzir_srt(Legenda_original_caminho, "legenda_traduzida.srt")

Traduzindo legendas: 100%|██████████| 1100/1100 [23:34<00:00,  1.29s/bloco]


✅ Tradução finalizada com sucesso!


# Multilingual Dubbing from Subtitle
### This is not accurate
Subtitle Dubbing Tool is a convenient utility designed to assist in the process of dubbing subtitles for videos. This tool streamlines the workflow by integrating audio upload, automatic subtitle generation, translation, and text-to-speech (TTS) capabilities.

## How to Use:

1. **Upload Audio:**
   - Begin by uploading the audio file corresponding to the video content you wish to dub.

2. **Whisper to Generate Subtitle (.srt) Format:**
   - Utilize the whisper feature to automatically generate subtitles in the .srt format. This feature transcribes the spoken content of the uploaded audio into text, ensuring accurate representation.

3. **Google Translate for Translation:**
   - Translate the generated subtitles into your desired language using Google Translate. This step ensures that your subtitles are accessible to a broader audience by providing translations in multiple languages.

4. **Edge TTS for Multilingual TTS:**
   - Employ Edge TTS (Text-to-Speech) functionality to convert the translated subtitles into speech. This enables the creation of dubbed audio tracks in various languages, enhancing the accessibility and reach of your video content.

## Usage Instructions:

- Follow the numbered steps listed above to sequentially navigate through the subtitle dubbing process.
- Ensure that the uploaded audio file is clear and of sufficient quality to facilitate accurate transcription and dubbing.
- Review and refine the generated subtitles and translations as necessary to maintain accuracy and coherence.
- Experiment with different languages and TTS voices offered by Edge TTS to customize the dubbing experience according to your preferences and audience demographics.

## Notes:

- Subtitle Dubbing Tool is intended to streamline the dubbing process and enhance the accessibility of video content by providing automated transcription, translation, and text-to-speech functionalities.
- While the tool aims to produce accurate results, it is advisable to review the generated subtitles and translations for any errors or discrepancies.
- Feedback and suggestions for improvement are welcome to continually enhance the functionality and usability of the tool.

Thank you for using Subtitle Dubbing Tool!

In [1]:
#@title Install
!pip install git+https://github.com/openai/whisper.git
!sudo apt update && sudo apt install ffmpeg
!pip install pydub
!pip install edge-tts
!pip install googletrans==3.1.0a0
!pip install pysrt
from IPython.display import clear_output
clear_output()

## If you don't have the srt File . First Generate the .srt File.

In [5]:
#@title Whisper Model load

import uuid
import string
def unique_file_name(text):
    if text.strip():
        random_string = str(uuid.uuid4())
        save_path = text[:20]
        save_path = save_path.translate(str.maketrans("", "", string.punctuation)).replace(" ", "_")
        return save_path  # Return the generated save_path
    else:
        return "dummy"

def format_time(seconds):
    minutes, seconds = divmod(seconds, 60)
    hours, minutes = divmod(minutes, 60)
    milliseconds = int((seconds - int(seconds)) * 1000)  # Extract milliseconds as an integer
    return f"{hours:02d}:{minutes:02d}:{int(seconds):02d},{milliseconds:03d}"

def generate_srt(result,save_srt):
    srt_content = ""

    for i, segment in enumerate(result['segments']):
        start_time = int(segment['start'])
        end_time = int(segment['end'])
        text = segment['text'].strip()

        srt_content += f"{i + 1}\n"
        srt_content += f"{format_time(start_time)} --> {format_time(end_time)}\n"
        srt_content += f"{text}\n\n"

    with open(save_srt, 'w') as file:
        file.write(srt_content)
    return save_srt

import os


import whisper
select_whisper_model ="base"  # @param ["tiny", "base", "small","medium","large"] {allow-input: true}
model = whisper.load_model(select_whisper_model)
import torch
gpu = torch.cuda.is_available()

def convert_to_text(audio_path):
  if gpu:
    result = model.transcribe(audio_path,word_timestamps=True,fp16=True)
  else:
    result = model.transcribe(audio_path,word_timestamps=True,fp16=False)
  with open('scan.txt', 'w') as file:
    file.write(str(result))
  if not os.path.exists("/content/whisper_subtitle"):
    os.mkdir("/content/whisper_subtitle")
  save_srt=f"/content/whisper_subtitle/{unique_file_name(result['text'].strip())}.srt"
  # save_srt=str(uuid.uuid4())[:8]+".srt"
  generate_srt(result,save_srt)
  return save_srt,result["text"].strip()
# convert_to_text("/content/up.MP3")

100%|████████████████████████████████████████| 139M/139M [00:00<00:00, 317MiB/s]


In [7]:
#@title Upload your audio
import os
from google.colab import files
import shutil

upload_folder = '/content/user_upload'

if not os.path.exists(upload_folder):
    os.mkdir(upload_folder)
target_person=[]
uploaded = files.upload()
for filename in uploaded.keys():
  dst_path = os.path.join(upload_folder, filename)
  print(f'move {filename} to {dst_path}')
  shutil.move(filename, dst_path)
  target_person.append(dst_path)

from IPython.display import clear_output
clear_output()
target_person[-1]


'/content/user_upload/Parte 2.mp3'

In [20]:
#@title Enter Audio Path

audio_path = "/content/user_upload/Parte 2.mp3"  # @param {type: "string"}
save_srt_path,_=convert_to_text(audio_path)
save_srt_path

'/content/whisper_subtitle/and_it_looks_nice_I.srt'

## If you already have the srt . Start from here.

Use DownaSub to generate Subtitle from youtube video <br>
[downsub](https://downsub.com/)

In [19]:
#@title Edge tts Config and demo
def calculate_rate_string(input_value):
    rate = (input_value - 1) * 100
    sign = '+' if input_value >= 1 else '-'
    return f"{sign}{abs(int(rate))}"
languages = {
    "Afrikaans": "af",
    "Amharic": "am",
    "Arabic": "ar",
    "Azerbaijani": "az",
    "Bulgarian": "bg",
    "Bengali": "bn",
    "Bosnian": "bs",
    "Catalan": "ca",
    "Czech": "cs",
    "Welsh": "cy",
    "Danish": "da",
    "German": "de",
    "Greek": "el",
    "English": "en",
    "Spanish": "es",
    "French": "fr",
    "Irish": "ga",
    "Galician": "gl",
    "Gujarati": "gu",
    "Hebrew": "he",
    "Hindi": "hi",
    "Croatian": "hr",
    "Hungarian": "hu",
    "Indonesian": "id",
    "Icelandic": "is",
    "Italian": "it",
    "Japanese": "ja",
    "Javanese": "jv",
    "Georgian": "ka",
    "Kazakh": "kk",
    "Khmer": "km",
    "Kannada": "kn",
    "Korean": "ko",
    "Lao": "lo",
    "Lithuanian": "lt",
    "Latvian": "lv",
    "Macedonian": "mk",
    "Malayalam": "ml",
    "Mongolian": "mn",
    "Marathi": "mr",
    "Malay": "ms",
    "Maltese": "mt",
    "Burmese": "my",
    "Norwegian Bokmål": "nb",
    "Nepali": "ne",
    "Dutch": "nl",
    "Polish": "pl",
    "Pashto": "ps",
    "Portuguese": "pt",
    "Romanian": "ro",
    "Russian": "ru",
    "Sinhala": "si",
    "Slovak": "sk",
    "Slovenian": "sl",
    "Somali": "so",
    "Albanian": "sq",
    "Serbian": "sr",
    "Sundanese": "su",
    "Swedish": "sv",
    "Swahili": "sw",
    "Tamil": "ta",
    "Telugu": "te",
    "Thai": "th",
    "Turkish": "tr",
    "Ukrainian": "uk",
    "Urdu": "ur",
    "Uzbek": "uz",
    "Vietnamese": "vi",
    "Chinese": "zh",
    "Zulu": "zu"
}



from googletrans import Translator

def translate_text(text, Language):
    target_language=languages[Language]
    # if Language == "English" :
    #   target_language='en'
    # if Language == "Hindi":
    #   target_language='hi'
    # if Language == "Bengali":
    #   target_language='bn'
    if Language == "Chinese":
          target_language='zh-CN'
    translator = Translator()
    translation = translator.translate(text, dest=target_language)
    t_text=translation.text
    if Language == "English" :
      return t_text
    elif Language == "Hindi" or Language == "Bengali":
      return t_text.replace(".","।")
    else:
      return t_text


def make_chunks(input_text, language):
    return [input_text]
    # if language == "English":
    #     temp_list = input_text.strip().split(".")
    #     filtered_list = [element.strip() + '.' for element in temp_list[:-1] if element.strip() and element.strip() != "'" and element.strip() != '"']
    #     if temp_list[-1].strip():
    #         filtered_list.append(temp_list[-1].strip())
    #     return filtered_list

    # elif language == "Hindi" or language == "Bengali":
    #     temp_list = input_text.strip().split("।")
    #     filtered_list = [element.strip() + '।' for element in temp_list[:-1] if element.strip() and element.strip() != "'" and element.strip() != '"']
    #     if temp_list[-1].strip():
    #         filtered_list.append(temp_list[-1].strip())
    #     return filtered_list
    # else:
    #   return [input_text]



import re
import uuid
def tts_file_name(text):
    if text.endswith("."):
        text = text[:-1]
    text = text.lower()
    text = text.strip()
    text = text.replace(" ","_")
    truncated_text = text[:25] if len(text) > 25 else text if len(text) > 0 else "empty"
    random_string = uuid.uuid4().hex[:8].upper()
    file_name = f"/content/edge_tts_voice/{truncated_text}_{random_string}.mp3"
    return file_name


from pydub import AudioSegment
import shutil
import os
def merge_audio_files(audio_paths, output_path):
    # Initialize an empty AudioSegment
    merged_audio = AudioSegment.silent(duration=0)

    # Iterate through each audio file path
    for audio_path in audio_paths:
        # Load the audio file using Pydub
        audio = AudioSegment.from_file(audio_path)

        # Append the current audio file to the merged_audio
        merged_audio += audio

    # Export the merged audio to the specified output path
    merged_audio.export(output_path, format="mp3")

def edge_free_tts(chunks_list,speed,voice_name,save_path):
  # voice_name="en-IE-EmilyNeural"  # @param {type: "string"}
  print(chunks_list)
  if len(chunks_list)>1:
    chunk_audio_list=[]
    if os.path.exists("/content/edge_tts_voice"):
      shutil.rmtree("/content/edge_tts_voice")
    os.mkdir("/content/edge_tts_voice")
    k=1
    for i in chunks_list:
      print(i)
      edge_command=f'''edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{text}" --write-media {save_path}'''
      # edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{i}" --write-media /content/edge_tts_voice/{k}.mp3'

      var1=os.system(edge_command)
      if var1==0:
        pass
      else:
        print(f"Failed: {i}")
        print(edge_command)
      chunk_audio_list.append(f"/content/edge_tts_voice/{k}.mp3")
      k+=1
    print(chunk_audio_list)
    merge_audio_files(chunk_audio_list, save_path)
  else:
    edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{chunks_list[0]}" --write-media {save_path}'
    print(edge_command)
    var2=os.system(edge_command)
    if var2==0:
      pass
    else:
      print(f"Failed: {chunks_list[0]}")
  return save_path
female_voice_list={'Vietnamese': 'vi-VN-HoaiMyNeural',
 'Bengali': 'bn-BD-NabanitaNeural',
 'Thai': 'th-TH-PremwadeeNeural',
 'English': 'en-AU-NatashaNeural',
 'Portuguese': 'pt-BR-FranciscaNeural',
 'Arabic': 'ar-AE-FatimaNeural',
 'Turkish': 'tr-TR-EmelNeural',
 'Spanish': 'es-AR-ElenaNeural',
 'Korean': 'ko-KR-SunHiNeural',
 'French': 'fr-BE-CharlineNeural',
 'Indonesian': 'id-ID-GadisNeural',
 'Russian': 'ru-RU-SvetlanaNeural',
 'Hindi': 'hi-IN-SwaraNeural',
 'Japanese': 'ja-JP-NanamiNeural',
 'Afrikaans': 'af-ZA-AdriNeural',
 'Amharic': 'am-ET-MekdesNeural',
 'Azerbaijani': 'az-AZ-BanuNeural',
 'Bulgarian': 'bg-BG-KalinaNeural',
 'Bosnian': 'bs-BA-VesnaNeural',
 'Catalan': 'ca-ES-JoanaNeural',
 'Czech': 'cs-CZ-VlastaNeural',
 'Welsh': 'cy-GB-NiaNeural',
 'Danish': 'da-DK-ChristelNeural',
 'German': 'de-AT-IngridNeural',
 'Greek': 'el-GR-AthinaNeural',
 'Irish': 'ga-IE-OrlaNeural',
 'Galician': 'gl-ES-SabelaNeural',
 'Gujarati': 'gu-IN-DhwaniNeural',
 'Hebrew': 'he-IL-HilaNeural',
 'Croatian': 'hr-HR-GabrijelaNeural',
 'Hungarian': 'hu-HU-NoemiNeural',
 'Icelandic': 'is-IS-GudrunNeural',
 'Italian': 'it-IT-ElsaNeural',
 'Javanese': 'jv-ID-SitiNeural',
 'Georgian': 'ka-GE-EkaNeural',
 'Kazakh': 'kk-KZ-AigulNeural',
 'Khmer': 'km-KH-SreymomNeural',
 'Kannada': 'kn-IN-SapnaNeural',
 'Lao': 'lo-LA-KeomanyNeural',
 'Lithuanian': 'lt-LT-OnaNeural',
 'Latvian': 'lv-LV-EveritaNeural',
 'Macedonian': 'mk-MK-MarijaNeural',
 'Malayalam': 'ml-IN-SobhanaNeural',
 'Mongolian': 'mn-MN-YesuiNeural',
 'Marathi': 'mr-IN-AarohiNeural',
 'Malay': 'ms-MY-YasminNeural',
 'Maltese': 'mt-MT-GraceNeural',
 'Burmese': 'my-MM-NilarNeural',
 'Norwegian Bokmål': 'nb-NO-PernilleNeural',
 'Nepali': 'ne-NP-HemkalaNeural',
 'Dutch': 'nl-BE-DenaNeural',
 'Polish': 'pl-PL-ZofiaNeural',
 'Pashto': 'ps-AF-LatifaNeural',
 'Romanian': 'ro-RO-AlinaNeural',
 'Sinhala': 'si-LK-ThiliniNeural',
 'Slovak': 'sk-SK-ViktoriaNeural',
 'Slovenian': 'sl-SI-PetraNeural',
 'Somali': 'so-SO-UbaxNeural',
 'Albanian': 'sq-AL-AnilaNeural',
 'Serbian': 'sr-RS-SophieNeural',
 'Sundanese': 'su-ID-TutiNeural',
 'Swedish': 'sv-SE-SofieNeural',
 'Swahili': 'sw-KE-ZuriNeural',
 'Tamil': 'ta-IN-PallaviNeural',
 'Telugu': 'te-IN-ShrutiNeural',
 'Chinese': 'zh-CN-XiaoxiaoNeural',
 'Ukrainian': 'uk-UA-PolinaNeural',
 'Urdu': 'ur-IN-GulNeural',
 'Uzbek': 'uz-UZ-MadinaNeural',
 'Zulu': 'zu-ZA-ThandoNeural'}
male_voice_list= {'Vietnamese': 'vi-VN-NamMinhNeural',
 'Bengali': 'bn-BD-PradeepNeural',
 'Thai': 'th-TH-NiwatNeural',
 'English': 'en-AU-WilliamNeural',
 'Portuguese': 'pt-BR-AntonioNeural',
 'Arabic': 'ar-AE-HamdanNeural',
 'Turkish': 'tr-TR-AhmetNeural',
 'Spanish': 'es-AR-TomasNeural',
 'Korean': 'ko-KR-HyunsuNeural',
 'French': 'fr-BE-GerardNeural',
 'Indonesian': 'id-ID-ArdiNeural',
 'Russian': 'ru-RU-DmitryNeural',
 'Hindi': 'hi-IN-MadhurNeural',
 'Japanese': 'ja-JP-KeitaNeural',
 'Afrikaans': 'af-ZA-WillemNeural',
 'Amharic': 'am-ET-AmehaNeural',
 'Azerbaijani': 'az-AZ-BabekNeural',
 'Bulgarian': 'bg-BG-BorislavNeural',
 'Bosnian': 'bs-BA-GoranNeural',
 'Catalan': 'ca-ES-EnricNeural',
 'Czech': 'cs-CZ-AntoninNeural',
 'Welsh': 'cy-GB-AledNeural',
 'Danish': 'da-DK-JeppeNeural',
 'German': 'de-AT-JonasNeural',
 'Greek': 'el-GR-NestorasNeural',
 'Irish': 'ga-IE-ColmNeural',
 'Galician': 'gl-ES-RoiNeural',
 'Gujarati': 'gu-IN-NiranjanNeural',
 'Hebrew': 'he-IL-AvriNeural',
 'Croatian': 'hr-HR-SreckoNeural',
 'Hungarian': 'hu-HU-TamasNeural',
 'Icelandic': 'is-IS-GunnarNeural',
 'Italian': 'it-IT-DiegoNeural',
 'Javanese': 'jv-ID-DimasNeural',
 'Georgian': 'ka-GE-GiorgiNeural',
 'Kazakh': 'kk-KZ-DauletNeural',
 'Khmer': 'km-KH-PisethNeural',
 'Kannada': 'kn-IN-GaganNeural',
 'Lao': 'lo-LA-ChanthavongNeural',
 'Lithuanian': 'lt-LT-LeonasNeural',
 'Latvian': 'lv-LV-NilsNeural',
 'Macedonian': 'mk-MK-AleksandarNeural',
 'Malayalam': 'ml-IN-MidhunNeural',
 'Mongolian': 'mn-MN-BataaNeural',
 'Marathi': 'mr-IN-ManoharNeural',
 'Malay': 'ms-MY-OsmanNeural',
 'Maltese': 'mt-MT-JosephNeural',
 'Burmese': 'my-MM-ThihaNeural',
 'Norwegian Bokmål': 'nb-NO-FinnNeural',
 'Nepali': 'ne-NP-SagarNeural',
 'Dutch': 'nl-BE-ArnaudNeural',
 'Polish': 'pl-PL-MarekNeural',
 'Pashto': 'ps-AF-GulNawazNeural',
 'Romanian': 'ro-RO-EmilNeural',
 'Sinhala': 'si-LK-SameeraNeural',
 'Slovak': 'sk-SK-LukasNeural',
 'Slovenian': 'sl-SI-RokNeural',
 'Somali': 'so-SO-MuuseNeural',
 'Albanian': 'sq-AL-IlirNeural',
 'Serbian': 'sr-RS-NicholasNeural',
 'Sundanese': 'su-ID-JajangNeural',
 'Swedish': 'sv-SE-MattiasNeural',
 'Swahili': 'sw-KE-RafikiNeural',
 'Tamil': 'ta-IN-ValluvarNeural',
 'Telugu': 'te-IN-MohanNeural',
 'Chinese': 'zh-CN-YunjianNeural',
 'Ukrainian': 'uk-UA-OstapNeural',
 'Urdu': 'ur-IN-SalmanNeural',
 'Uzbek': 'uz-UZ-SardorNeural',
 'Zulu': 'zu-ZA-ThembaNeural'}
text = 'Hi, How are you .'  # @param {type: "string"}
Language = "Portuguese" # @param ['Afrikaans', 'Amharic', 'Arabic', 'Azerbaijani', 'Bulgarian', 'Bengali', 'Bosnian', 'Catalan', 'Czech', 'Welsh', 'Danish', 'German', 'Greek', 'English', 'Spanish', 'French', 'Irish', 'Galician', 'Gujarati', 'Hebrew', 'Hindi', 'Croatian', 'Hungarian', 'Indonesian', 'Icelandic', 'Italian', 'Japanese', 'Javanese', 'Georgian', 'Kazakh', 'Khmer', 'Kannada', 'Korean', 'Lao', 'Lithuanian', 'Latvian', 'Macedonian', 'Malayalam', 'Mongolian', 'Marathi', 'Malay', 'Maltese', 'Burmese', 'Norwegian Bokmål', 'Nepali', 'Dutch', 'Polish', 'Pashto', 'Portuguese', 'Romanian', 'Russian', 'Sinhala', 'Slovak', 'Slovenian', 'Somali', 'Albanian', 'Serbian', 'Sundanese', 'Swedish', 'Swahili', 'Tamil', 'Telugu', 'Thai', 'Turkish', 'Ukrainian', 'Urdu', 'Uzbek', 'Vietnamese', 'Chinese', 'Zulu']
Gender = "Male"# @param ['Male', 'Female']
speed = 1  # @param {type: "number"}

translate_text_flag  = True # @param {type:"boolean"}
# long_sentence = True # @param {type:"boolean"}
long_sentence=False
save_path = '/content/edge.wav'  # @param {type: "string"}
if len(save_path)==0:
  save_path=tts_file_name(text)
if Language == "English" :
  if Gender=="Male":
    # voice_name="en-US-ChristopherNeural"
    voice_name="en-US-BrianNeural"
  if Gender=="Female":
    voice_name="en-US-AriaNeural"
elif Language == "Hindi":
  if Gender=="Male":
    voice_name="hi-IN-MadhurNeural"
  if Gender=="Female":
    voice_name="hi-IN-SwaraNeural"
elif Language == "Bengali":
  if Gender=="Male":
    voice_name="bn-IN-BashkarNeural"
  if Gender=="Female":
    voice_name="bn-BD-NabanitaNeural"
else:
  if Gender=="Male":
    voice_name=male_voice_list[Language]
  if Gender=="Female":
    voice_name=female_voice_list[Language]
if translate_text_flag:
  input_text=translate_text(text, Language)
  print("Translateting")
else:
  input_text=text
if long_sentence==True and translate_text_flag==True:
  chunks_list=make_chunks(input_text,Language)
elif long_sentence==True and translate_text_flag==False:
  chunks_list=make_chunks(input_text,"English")
else:
  chunks_list=[input_text]
# print(chunks_list)
# print(chunks_list,speed,voice_name,save_path)
edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)





# remove_slience  = True # @param {type:"boolean"}
# slience_margin = 0.1  # @param {type: "number"}
remove_slience  = True
if remove_slience:
  new_file_path=edge_save_path
  # new_file_path,_=remove_silence_from_audio(edge_save_path, slience_margin)
else:
  new_file_path=edge_save_path
auto_download  = False # @param {type:"boolean"}
from google.colab import files
if auto_download:
  files.download(new_file_path)
from IPython.display import clear_output
clear_output()


def process_tts(text,speed,audio_path,Language,Gender,long_sentence,translate_text_flag):
  if Gender=="Male":
    voice_name=male_voice_list[Language]
  if Gender=="Female":
    voice_name=female_voice_list[Language]
  if translate_text_flag:
    input_text=translate_text(text, Language)
    print("Translateting")
  else:
    input_text=text
  if long_sentence==True and translate_text_flag==True:
    chunks_list=make_chunks(input_text,Language)
  elif long_sentence==True and translate_text_flag==False:
    chunks_list=make_chunks(input_text,"English")
  else:
    chunks_list=[input_text]
  edge_free_tts(chunks_list,speed,voice_name,audio_path)
# text="hi how are you"
# speed=1
# audio_path='/content/test.mp3'
# Language='English'
# Gender='Male'
# long_sentence=True
# translate_text_flag=True
# process_tts(text,speed,audio_path,Language,Gender,long_sentence,translate_text_flag)
# Audio(audio_path, autoplay=True)





from IPython.display import Audio
Audio(new_file_path, autoplay=True)

In [7]:
#@title Clean bad srt (Modify this according to your use)
import pysrt
Enter_srt_file_path = '/content/whisper_subtitle/and_it_looks_nice_I.srt'  # @param {type: "string"}
def clean_srt_line(text):
  bad_list=["[","]","♫","\n"]
  for i in bad_list:
    text=text.replace(i,"")
  return text.strip()
# Load the subtitle file
subs = pysrt.open(Enter_srt_file_path)
colab_srt_path="/content/clean.srt"
# Iterate through each subtitle and print its details
with open(colab_srt_path, "w", encoding='utf-8') as file:
  for sub in subs:
      # print(f'Index: {sub.index}')
      # print(f'Start: {sub.start}')
      # print(f'End: {sub.end}')
      # print(f'Text: {sub.text}')
      file.write(f"{sub.index}\n")
      file.write(f"{sub.start} --> {sub.end}\n")
      file.write(f"{clean_srt_line(sub.text)}\n")
      file.write(f"\n")
  file.close()
print("Clean Srt save at:")
colab_srt_path

Clean Srt save at:


'/content/clean.srt'

## If your subtitle already in Given Language uncheck ```translate_text_flag```



In [22]:
#@title Dub Subtitle in Different Languages

def process_tts(text,speed,audio_path,Language,Gender,long_sentence,translate_text_flag):
  if Gender=="Male":
    voice_name=male_voice_list[Language]
  if Gender=="Female":
    voice_name=female_voice_list[Language]
  if translate_text_flag:
    input_text=translate_text(text, Language)
    print("Translateting")
  else:
    input_text=text
  if long_sentence==True and translate_text_flag==True:
    chunks_list=make_chunks(input_text,Language)
  elif long_sentence==True and translate_text_flag==False:
    chunks_list=make_chunks(input_text,"English")
  else:
    chunks_list=[input_text]
  edge_free_tts(chunks_list,speed,voice_name,audio_path)

import os
def get_subtitle_Dub_path(srt_file_path,Language):
  file_name = os.path.splitext(os.path.basename(srt_file_path))[0]
  if not os.path.exists("/content/TTS_DUB"):
    os.mkdir("/content/TTS_DUB")
  new_path=f"/content/TTS_DUB/{Language}_{file_name}.wav"
  return new_path
###

from pydub import AudioSegment
import shutil
import subprocess
import os
import uuid
import re


srt_file_path = '/content/Make your first Aimbot and ESP! C#⁄.NET Course Part 2 - Reversing structures (1080p_60fps_H264-128kbit_AAC).inglês.srt'  # @param {type: "string"}
# dub_save_path = '/content/final.wav' # @param {type: "string"}
Language = "Portuguese" # @param ['Afrikaans', 'Amharic', 'Arabic', 'Azerbaijani', 'Bulgarian', 'Bengali', 'Bosnian', 'Catalan', 'Czech', 'Welsh', 'Danish', 'German', 'Greek', 'English', 'Spanish', 'French', 'Irish', 'Galician', 'Gujarati', 'Hebrew', 'Hindi', 'Croatian', 'Hungarian', 'Indonesian', 'Icelandic', 'Italian', 'Japanese', 'Javanese', 'Georgian', 'Kazakh', 'Khmer', 'Kannada', 'Korean', 'Lao', 'Lithuanian', 'Latvian', 'Macedonian', 'Malayalam', 'Mongolian', 'Marathi', 'Malay', 'Maltese', 'Burmese', 'Norwegian Bokmål', 'Nepali', 'Dutch', 'Polish', 'Pashto', 'Portuguese', 'Romanian', 'Russian', 'Sinhala', 'Slovak', 'Slovenian', 'Somali', 'Albanian', 'Serbian', 'Sundanese', 'Swedish', 'Swahili', 'Tamil', 'Telugu', 'Thai', 'Turkish', 'Ukrainian', 'Urdu', 'Uzbek', 'Vietnamese', 'Chinese', 'Zulu']
dub_save_path=get_subtitle_Dub_path(srt_file_path,Language)
import time
def your_tts(text,audio_path,language):
  Language=language
  Gender = "Male"# @param ['Male', 'Female']
  speed = 1  # @param {type: "number"}
  # long_sentence=True #@param {type:"boolean"}
  long_sentence=False
  translate_text_flag=False# @param {type:"boolean"}
  process_tts(text,speed,audio_path,Language,Gender,long_sentence,translate_text_flag)
  if long_sentence:
    time.sleep(1)

class SRTDubbing:
    def __init__(self):
        pass

    @staticmethod
    def text_to_speech(text, audio_path, language, actual_duration):
        tts_filename = "temp.wav"
        your_tts(text,tts_filename,language)
        # Check the duration of the generated TTS audio
        tts_audio = AudioSegment.from_file(tts_filename)
        tts_duration = len(tts_audio)

        if actual_duration == 0:
            # If actual duration is zero, use the original TTS audio without modifications
            shutil.move(tts_filename, audio_path)
            return

        # If TTS audio duration is longer than actual duration, speed up the audio
        if tts_duration > actual_duration:
            speedup_factor = tts_duration / actual_duration
            speedup_filename = "speedup_temp.wav"

            # Use ffmpeg to change audio speed
            subprocess.run([
                "ffmpeg",
                "-i", tts_filename,
                "-filter:a", f"atempo={speedup_factor}",
                speedup_filename
            ], check=True)

            # Replace the original TTS audio with the sped-up version
            shutil.move(speedup_filename, audio_path)
        elif tts_duration < actual_duration:
            # If TTS audio duration is less than actual duration, add silence to match the duration
            silence_gap = actual_duration - tts_duration
            silence = AudioSegment.silent(duration=int(silence_gap))
            new_audio = tts_audio + silence

            # Save the new audio with added silence
            new_audio.export(audio_path, format="wav")
        else:
            # If TTS audio duration is equal to actual duration, use the original TTS audio
            shutil.move(tts_filename, audio_path)

    @staticmethod
    def make_silence(pause_time, pause_save_path):
        silence = AudioSegment.silent(duration=pause_time)
        silence.export(pause_save_path, format="wav")
        return pause_save_path

    @staticmethod
    def create_folder_for_srt(srt_file_path):
        srt_base_name = os.path.splitext(os.path.basename(srt_file_path))[0]
        random_uuid = str(uuid.uuid4())[:4]
        dummy_folder_path = "/content/dummy"
        if not os.path.exists(dummy_folder_path):
            os.makedirs(dummy_folder_path)
        folder_path = os.path.join(dummy_folder_path, f"{srt_base_name}_{random_uuid}")
        os.makedirs(folder_path, exist_ok=True)
        return folder_path

    @staticmethod
    def concatenate_audio_files(audio_paths, output_path):
        concatenated_audio = AudioSegment.silent(duration=0)
        for audio_path in audio_paths:
            audio_segment = AudioSegment.from_file(audio_path)
            concatenated_audio += audio_segment
        concatenated_audio.export(output_path, format="wav")

    def srt_to_dub(self, srt_file_path, dub_save_path,language='en'):
        result = self.read_srt_file(srt_file_path)
        new_folder_path = self.create_folder_for_srt(srt_file_path)
        join_path = []
        for i in result:
            text = i['text']
            actual_duration = i['end_time'] - i['start_time']
            pause_time = i['pause_time']
            slient_path = f"{new_folder_path}/{i['previous_pause']}"
            self.make_silence(pause_time, slient_path)
            join_path.append(slient_path)
            tts_path = f"{new_folder_path}/{i['audio_name']}"
            self.text_to_speech(text, tts_path, language, actual_duration)
            join_path.append(tts_path)
        self.concatenate_audio_files(join_path, dub_save_path)

    @staticmethod
    def convert_to_millisecond(time_str):
      if isinstance(time_str, str):
          hours, minutes, second_millisecond = time_str.split(':')
          seconds, milliseconds = second_millisecond.split(",")

          total_milliseconds = (
              int(hours) * 3600000 +
              int(minutes) * 60000 +
              int(seconds) * 1000 +
              int(milliseconds)
          )

          return total_milliseconds
    @staticmethod
    def read_srt_file(file_path):
        entries = []
        default_start = 0
        previous_end_time = default_start
        entry_number = 1
        audio_name_template = "{}.wav"
        previous_pause_template = "{}_before_pause.wav"

        with open(file_path, 'r', encoding='utf-8') as file:
            lines = file.readlines()
            print(lines)
            for i in range(0, len(lines), 4):
                time_info = re.findall(r'(\d+:\d+:\d+,\d+) --> (\d+:\d+:\d+,\d+)', lines[i + 1])
                start_time = SRTDubbing.convert_to_millisecond(time_info[0][0])
                end_time = SRTDubbing.convert_to_millisecond(time_info[0][1])

                current_entry = {
                    'entry_number': entry_number,
                    'start_time': start_time,
                    'end_time': end_time,
                    'text': lines[i + 2].strip(),
                    'pause_time': start_time - previous_end_time if entry_number != 1 else start_time - default_start,
                    'audio_name': audio_name_template.format(entry_number),
                    'previous_pause': previous_pause_template.format(entry_number),
                }

                entries.append(current_entry)
                previous_end_time = end_time
                entry_number += 1

        return entries

# Example usage
srt_dubbing = SRTDubbing()
current_language=Language
srt_dubbing.srt_to_dub(srt_file_path, dub_save_path,current_language)
from IPython.display import clear_output
clear_output()
print(f"{Language} Dub Audio File Save At : {dub_save_path}")
from google.colab import files

files.download(dub_save_path)




['1\n', '00:00:03,339 --> 00:00:17,920\n', '[Music]\n', '\n', '2\n', '00:00:14,480 --> 00:00:20,800\n', 'all right so welcome to part two of the\n', '\n', '3\n', '00:00:17,920 --> 00:00:23,199\n', 'esp and aimbot series\n', '\n', '4\n', '00:00:20,800 --> 00:00:25,199\n', 'in this part we will find the memory\n', '\n', '5\n', '00:00:23,199 --> 00:00:26,800\n', 'addresses offsets\n', '\n', '6\n', '00:00:25,199 --> 00:00:30,000\n', 'pointers\n', '\n', '7\n', '00:00:26,800 --> 00:00:33,039\n', 'everything reversing the structures of\n', '\n', '8\n', '00:00:30,000 --> 00:00:34,960\n', 'assault cube to use for our aimbot and\n', '\n', '9\n', '00:00:33,039 --> 00:00:36,640\n', 'esp so\n', '\n', '10\n', '00:00:34,960 --> 00:00:38,480\n', 'the information that we will need to\n', '\n', '11\n', '00:00:36,640 --> 00:00:41,759\n', 'find\n', '\n', '12\n', '00:00:38,479 --> 00:00:42,959\n', 'i can show you guys in the final project\n', '\n', '13\n', '00:00:41,759 --> 00:00:45,199\n', 'is\n', '\n', '

JSONDecodeError: Expecting value: line 1 column 1 (char 0)